In [1]:
# how to kill all processes
# sudo kill -9 $(pgrep -f python3)
# sudo kill -9 $(pgrep -f conformanceTests)

# import subprocess
# subprocess.run(["kill" "-9" $(pgrep -f conformanceTests)"])
import os
os.system("kill -9 $(pgrep -f conformanceTests)")

9

In [14]:
!pip install xlsxwriter==3.0.2
!pip install beautifulsoup4==4.10.0
!pip install utils==1.0.1

In [182]:
import os
python_exec = '/home/nsemaev/code/venv/bin/python3.8'

work_path = os.getcwd()

binary_type = 'master_2022_02_15'
binary_name = f"conformanceTests_{binary_type}"
binary_path = f"{work_path}/bin/{binary_name}"


ci_report_file = f"{work_path}/ci_reports/" + "2022_02_15.html"
report_plugin_position = 4

results_path = f"{work_path}/{binary_type}"

ping_time = 60 * 5

cores = 16

irs_path = '/home/nsemaev/Documents/ops/'

skipped_ops = ['boolean', 'ROIAlign']
time_wasting_ops = ['BinaryConvolution', 'Convolution', 'ConvolutionBackpropData', 'DeformableConvolution', 
                    'GroupConvolution', 'GroupConvolutionBackpropData', 'MaxPool', 'AvgPool', 'Interpolate',
                    'TensorIterator', 'MVN', 'Reshape']

all_ops = sorted(os.listdir(irs_path))

run_results = ['passed', 'failed', 'skipped', 'crashed', 'empty']
ci_results = ['passed', 'failed', 'skipped', 'crashed']

In [189]:
def get_precision_irs_list(operation):
    precision_irs_dict = [{} for i in range(10)]
    for precision in os.listdir(f"{irs_path}/{operation}"):
        assert os.path.isdir(f"{irs_path}/{operation}/{precision}")
        for file_name in os.listdir(f"{irs_path}/{operation}/{precision}"):
            _, file_extension = os.path.splitext(file_name)
            if file_extension == '.xml':
                ir_id = int(re.findall(fr"{operation}_(\d+)\.xml", file_name)[0])
                with open(f"{irs_path}/{operation}/{precision}/{file_name}") as xml_file:
                    xml_data = str(xml_file.read())
                opset = int(re.findall(fr'type="{operation.lower()}" version="opset(\d+)">', xml_data.lower())[0])
                if precision not in precision_irs_dict[opset]:
                    precision_irs_dict[opset][precision] = []
#                 print(opset, type(opset))
                precision_irs_dict[opset][precision].append(ir_id)
    return precision_irs_dict

In [190]:
get_precision_irs_list('Add')

[{},
 {'f16': [592109, 206486, 223903, 1345, 50056, 1269, 1395938, 1395935],
  'f32': [15647, 69713, 42407, 712028, 579, 175242, 712025, 655],
  'i64': [687724, 2240, 67575, 67590],
  'i32': [42965, 1426552, 1767779, 1060907]},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {}]

In [200]:
import os
from os import path
import sys
import time
from datetime import datetime
import shutil
import subprocess
import _thread
import threading
from contextlib import contextmanager
import re
import xlsxwriter
from bs4 import BeautifulSoup


completed_ops = []
if os.path.exists(results_path):
    completed_ops = [op.split('_')[0] for op in os.listdir(results_path) if op.split('_')[-1] == 'completed']
logs_files = {}

class GTestParallel():
    def __init__(self, operation: str):
        print(f"\n\n\n{operation}")
        self.operation = operation
        self.op_results_path = f"{results_path}/{self.operation}_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}"
        self.precision_irs_list = get_precision_irs_list(self.operation)
#         self.reports_path = f"{self.op_results_path}_reports"
#         os.makedirs(self.reports_path)
        for opset in range(len(self.precision_irs_list)):
            for precision in self.precision_irs_list[opset]:
                for ir_id in self.precision_irs_list[opset][precision]:
                    dst_folder = f"{self.operation}_opset{opset}_{precision}_{ir_id}"
                    dst_path = f"{self.op_results_path}/{dst_folder}/ir"
                    os.makedirs(dst_path)
                    for extension in ['xml', 'bin', 'meta']:
                        ir_file_name = f"{self.operation}_{ir_id}.{extension}"
                        src_path = f"{irs_path}/{self.operation}/{precision}"
                        shutil.copyfile(f"{src_path}/{ir_file_name}", f"{dst_path}/{ir_file_name}")
               
    def get_command(self, folder: str):
        return f"{python_exec} {work_path}/gtest_parallel.py {binary_path} -d . " + \
            f"--gtest_filter=*ReadIRTest*{self.operation}* " + \
            f"-- " + \
            f"--input_folders={self.op_results_path}/{folder}/ir " + \
            f"--device=TEMPLATE " + \
            f"--output_folder={self.op_results_path}/{folder}"
#             f"--report_unique_name " + \
#             f"--output_folder={self.op_results_path}/{folder}/gtest-parallel-xmls"
    
    def run (self, time_limit):
        completed = 0
        start_time = time.time()
        waiting_time = 10
        for folder in os.listdir(self.op_results_path):
            command = self.get_command(folder)
            threading.Thread(target=os.system, args=(f"cd {self.op_results_path}/{folder} && {command}",)).start()
        while completed < len(os.listdir(self.op_results_path)):
            for folder in os.listdir(self.op_results_path):
                if not folder.endswith('completed'):
                    if os.path.exists(f"{self.op_results_path}/{folder}/gtest-parallel-logs/passed") or \
                       os.path.exists(f"{self.op_results_path}/{folder}/gtest-parallel-logs/failed"):
#                         report_name = f"{folder}_report.xml"
#                         if os.path.exists(f"{self.op_results_path}/{folder}/report.xml"):
#                             os.rename(f"{self.op_results_path}/{folder}/report.xml", 
#                                       f"{self.op_results_path}/{folder}/{report_name}")
#                             shutil.copyfile(f"{self.op_results_path}/{folder}/{report_name}", 
#                                             f"{self.reports_path}/{report_name}")
                        os.rename(f"{self.op_results_path}/{folder}", 
                                  f"{self.op_results_path}/{folder}_{int(time.time() - start_time)}s_completed")
                        completed += 1
            if datetime.now().strftime('%Y_%m_%d %H:%M:%S')[-1] == '0':
                print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.operation} ({time.time() - start_time} of {time_limit})")
                print(f"{completed} of {len(os.listdir(self.op_results_path))}")
            if time.time() - start_time > time_limit:
                os.system('kill -9 $(pgrep -f conformanceTests)')
                break
            time.sleep(1)
        os.rename(f"{self.op_results_path}", 
                  f"{self.op_results_path}_{completed}_of_{len(os.listdir(self.op_results_path))}_{int(time.time() - start_time)}s_completed")
    
    def get_precision_irs_dict(self):
        for precision in os.listdir(f"{irs_path}/{self.op}/"):
            assert os.path.isdir(f"{irs_path}/{self.op}/{precision}/")
            for file_name in os.listsir(f"{irs_path}/{self.op}/{precision}/"):
                _, file_extension = os.path.splitext(file_name)
        
    def generate_tmp_irs_path(self, irs_path: str, ir_id: int):
        os.makedirs(f"{work_path}/tmp/{self.op}_{ir_id}")
        shutil.copyfile(f"{irs_path}/", dst)
        
    

#     def run_while_not_end(self, op_time=30, time_limited=True):
#         process_start_time = time.time()
#         current_time = datetime.now().strftime('%Y_%m_%d %H:%M:%S')
#         print(f"{current_time} {self.op} was started")
#         print(self.command)
#         if os.path.exists(self.op_path):
#             shutil.rmtree(self.op_path)
#         os.makedirs(self.op_path)
#         os.chdir(self.op_path)
#         threading.Thread(target=os.system, args=(f"cd {self.op_path} && {self.command} > /dev/null",)).start()
#         # _thread.start_new_thread(os.system, (f"cd {self.op_path} && nohup {self.command}",))
#         os.chdir(work_path)
#         # statuses = ['passed', 'failed']
#         start_time = time.time()
#         last_ping = time.time()
#         while True:
#             time.sleep(1)
#             if (os.path.exists(f"{self.op_path}/gtest-parallel-logs/passed")):
#                 break
#             if (os.path.exists(f"{self.op_path}/gtest-parallel-logs/failed")):
#                 break
#             if time_limited and (time.time() - start_time > op_time):
#                 print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} Sorry, time for operation {self.op} is over")
#                 os.rename(self.op_path, self.op_stopped_path)
#                 os.system('kill -9 $(pgrep -f conformanceTests)')
#                 return False
#             if time.time() - last_ping > ping_time:
#                 print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.op} ping")
#                 last_ping = time.time()
        
#         log_files_folder = f"{self.op_path}/gtest-parallel-logs/failed"
#         failed_logs_result = f"{self.op_path}/{self.op}_failed_logs_result.txt"
#         if os.path.exists(log_files_folder):
#             with open(failed_logs_result, 'w', encoding='utf-8') as result_file:
#                 for log_file_name in os.listdir(log_files_folder):
#                     log_file_path = os.path.join(log_files_folder, log_file_name)  # !!! везде так же сделать

#                     # print(f"\n\n{log_file_path}\n\n")

#                     with open(log_file_path, 'r', encoding='utf-8') as file:
#                         data = str(file.read())
#                         test_filter = '/'.join(re.findall(r"Google Test filter = ([^\n]+)", data))
#                         mem_usage = '/'.join(re.findall(r"MEM_USAGE=([^\n]+)", data))
#                         failures = '/'.join(re.findall(r"[^\n]+pp:\s*\d+[^\n]+", data))
#                         # print(f"\n\n{test_filter}")
#                         # print(f"\n\n{mem_usage}")
#                         # print(f"\n\n{failures}")
#                         result_file.write(test_filter + ',')
#                         result_file.write(mem_usage + ',')
#                         result_file.write(failures + ';\n')
#                     # with open(log_path, 'r', encoding='utf-8') as file:
#                     #     print(file.read())

#         print(f"cd {self.op_path} && {self.report_command}")
#         threading.Thread(target=os.system, args=(f"cd {self.op_path} && {self.report_command}",)).start()
#         while (not os.path.exists(f"{self.op_path}/report.xml")):
#             pass
#         print(f"{datetime.now().strftime('%Y_%m_%d %H:%M:%S')} {self.op} was completed")
#         self.op_completed_path = f"{self.op_path}_{int(time.time() - process_start_time)}s_completed"
#         os.rename(self.op_path, self.op_completed_path)
#         return True



def generate_ci_data():
    data = {}
    with open(ci_report_file, 'r') as file:
        soup = BeautifulSoup(str(file.read()), 'html.parser')
    tbody = soup.findAll('tbody')[1]
    for op_tr in tbody.findAll('tr'):
        op = op_tr.findAll('th')[0].text.replace('-', '_opset')
        template_td = op_tr.findAll('td')[report_plugin_position]
        data[op] = {result: 'untested' for result in ci_results}
        for result in data[op].keys():
            if len(template_td.find_all('span')) == 4:
                data[op][result] = int(re.findall(fr"{result[0].upper()}:(\d+)", template_td.text)[0])
    return data

# def get_from_TensorIterator_report(op: str):
#     result = 'TensorIterator not tested'
#     folders = [folder for folder in os.listdir(results_path) 
#         if folder.split('_')[0] == 'TensorIterator' and folder.split('_')[-1] == 'completed']
#     if folders:
#         report_path = f"{results_path}/{sorted(folders)[-1]}/report.xml"
#         with open(report_path, 'r', encoding='utf-8') as file:
#             re_str = rf'<{op}-\d+ passed="\d+" failed="\d+" skipped="\d+" crashed="\d+" passrate="\d+\.\d+" />'
#             result = ''.join(re.findall(re_str, file.read()))    
#     return result

def not_all_ops():
    # TODO: Delete operations that have 100% passrate in both lists, a better algorithm
    bad_ops = []
    for op in ops:
        flag = False
        if op in run_data:
            for result in run_results:
                if result not in ['passed']:
                    if run_data[op][result] != 'untested' and int(run_data[op][result]) > 0:
                        flag = True
                        break
        if op in ci_data:
            for result in ci_results:
                if result not in ['passed']:
                    if ci_data[op][result] != 'untested' and int(ci_data[op][result]) > 0:
                        flag = True
                        break
        if flag:
            bad_ops.append(op)
    ops = bad_ops

def generate_xlsx():
    columns = ['Operation', 'Info', 'Group'] + \
                [f"RUN {result}" for result in run_results] + \
                [f"CI {result}" for result in ci_results] + ['RUN logs']
    columns_len = [len(column) for column in columns]
    run_data = generate_run_data()
    ci_data = generate_ci_data()
    ops = sorted(list(set(list(run_data.keys()) + list(ci_data.keys()))))
    for op in ops:
        if op in ci_data and op not in run_data and op.split('_')[0] in time_wasting_ops:
            print(op)
            run_data[op] = {result: 'TIME OUT' for result in run_results}
            

    # not_all_ops()

    workbook_path = f"{results_path}/table_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.xlsx"
    workbook = xlsxwriter.Workbook(workbook_path)
    worksheet = workbook.add_worksheet('report')
    for i, column in enumerate(columns):
        worksheet.write(0, i, column, workbook.add_format({"bold": True}))
    for i, op in enumerate(ops):
        shift = 0
        worksheet.write(i + 1, shift, op)
        columns_len[shift] = max(columns_len[shift], len(str(op)))
#         shift += 1
#         data = get_from_TensorIterator_report(op)
#         if data is not None:
#             worksheet.write(i + 1, shift, data)
        shift += 3
        if op in run_data:
            for j, result in enumerate(run_results):
                worksheet.write(i + 1, j + shift, run_data[op][result])
        shift += len(run_results)
        if op in ci_data:
            for j, result in enumerate(ci_results):
                worksheet.write(i + 1, j + shift, ci_data[op][result])
        shift += len(ci_results)
        if op in logs_files:
            with open(logs_files[op], 'r', encoding='utf-8') as file:
                worksheet.write(i + 1, shift, ''.join(file.readlines()))


    for i, column_len in enumerate(columns_len):
        if column_len > 0:
            worksheet.set_column(i, i, column_len)
    workbook.close()

def generate_full_report():
    if not os.path.exists(results_path):
        return
    folders = [folder for folder in os.listdir(results_path) if folder.split('_')[-1] == 'completed']
    folders = sorted(folders)
    ops = [folder.split('_')[0] for folder in folders]
    unique_folders, unique_ops = [], []
    for i, folder in enumerate(folders):
        if ops[i] not in ops[i+1:]:
            unique_folders.append(folder)
            unique_ops.append(ops[i])
    reports_path = f"{results_path}/reports"
    if os.path.exists(reports_path):
        shutil.rmtree(reports_path)
    os.makedirs(reports_path)
    for i, folder in enumerate(unique_folders):
        shutil.copy(f"{results_path}/{folder}/report.xml", f"{reports_path}/report_{unique_ops[i]}.xml")
    full_report_name = f"full_report_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}_{len(unique_ops)}pcs.xml"
    gen_report_command = f"{python_exec} {work_path}/merge_xmls.py -i {reports_path} -o {results_path}"
    os.system(gen_report_command)
    os.rename(f"{results_path}/report.xml", f"{results_path}/{full_report_name}")
    shutil.rmtree(reports_path)
    return f"{results_path}/{full_report_name}"
    
# TODO: get report_file by last name in work path
def generate_run_data():
    data = {}
    for i, op in enumerate(all_ops):
        run_folders = [folder for folder in os.listdir(results_path) 
                    if re.findall(fr"{op}_.+_completed", folder)]
#         data[op] = {result: 'untested' for result in run_results}
        if run_folders:
            run_path = f"{results_path}/{sorted(run_folders)[-1]}"
            for ir_run_folder in os.listdir(run_path):
                op_opset = re.findall(fr"{op}_opset\d+", ir_run_folder)[0]
                if op_opset not in data:
                    data[op_opset] = {result: 0 for result in run_results}
                report_path = f"{run_path}/{ir_run_folder}/report.xml"
                if os.path.exists(report_path):
                    with open(report_path, 'r') as report_file:
                        xml_data = str(report_file.read())
                    passed_pattern = fr'<{op}-\d+ implemented=".+" passed="1" failed="0" skipped="0" crashed="0"'
                    failed_pattern = fr'<{op}-\d+ implemented=".+" passed="0" failed="1" skipped="0" crashed="0"'
                    skipped_pattern = fr'<{op}-\d+ implemented=".+" passed="0" failed="0" skipped="1" crashed="0"'
                    crashed_pattern = fr'<{op}-\d+ implemented=".+" passed="0" failed="0" skipped="0" crashed="1"'
                    if re.findall(passed_pattern, xml_data):
                        data[op_opset]['passed'] += 1
                    elif re.findall(failed_pattern, xml_data):
                        data[op_opset]['failed'] += 1
                    elif re.findall(skipped_pattern, xml_data):
                        data[op_opset]['skipped'] += 1
                    elif re.findall(crashed_pattern, xml_data):
                        data[op_opset]['crashed'] += 1
                    else:
                        data[op_opset]['empty'] += 1
                else:
                    data[op_opset]['empty'] += 1
                    #print(f"REPORT??? {report_path}") # change
#             data[op] = {}
#             for result in run_results:
#                 result_path = f"{run_path}/gtest-parallel-logs/{result}"
#                 data[op][result] = len(os.listdir(result_path)) if path.exists(result_path) else 0

#             failed_report = f"{run_path}/{op}_failed_logs_result.txt"
#             if path.exists(failed_report):
#                 logs_files[op] = failed_report
    return data

if __name__ == '__main__':
    ops = [op for op in all_ops if op not in skipped_ops + completed_ops + time_wasting_ops]
#     ops = ['Add', 'Abs']
#     ops = []
    ops = time_wasting_ops
    print(f"Ops for run: {ops}")
    for op in ops:
#         GTestParallel(op).run_while_not_end(time_limited=False)
        GTestParallel(op).run(240)
#     time.sleep(5)
    print('End')
#     data = generate_ci_data()
#     data = generate_run_data()
#     for op, d in data.items():
#         if type(d['passed']) == int and d['passed'] > 0:
#             print(op)
#     print(data)
    
#     generate_full_report()
    generate_xlsx()
    
#         generate_xlsx()
#     generate_xlsx()
        

    # print(generate_run_data())
    # print(generate_ci_data())
    # generate_xlsx()


Ops for run: ['BinaryConvolution', 'Convolution', 'ConvolutionBackpropData', 'DeformableConvolution', 'GroupConvolution', 'GroupConvolutionBackpropData', 'MaxPool', 'AvgPool', 'Interpolate', 'TensorIterator', 'MVN', 'Reshape']



BinaryConvolution
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3011314.xml_TargetDevice=TEMPLATE_Config=() (3076 ms)
Note: Google Test filter = conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3011314.xml_TargetDevice=TEMPLATE_Config=()
[==========] Running 1 test from 1 test suite.
[----------] Global test environment set-up.
[----------] 1 test from conformance/ReadIRTest
[ RUN      ] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3011314.xml_TargetDevice=TEMPLATE_Config=()

MEM_USAGE=62488KB
Access PluginCache ov core. OV Core use count: 0
Created ov core.
Unexpected application crash with code: 11
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3011314.xml_Tar

2022_02_16 00:29:40 BinaryConvolution (10.708064556121826 of 240)
9 of 160
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247057.xml_TargetDevice=TEMPLATE_Config=() (8405 ms)
Note: Google Test filter = conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247057.xml_TargetDevice=TEMPLATE_Config=()
[==========] Running 1 test from 1 test suite.
[----------] Global test environment set-up.
[----------] 1 test from conformance/ReadIRTest
[ RUN      ] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247057.xml_TargetDevice=TEMPLATE_Config=()

MEM_USAGE=62488KB
Access PluginCache ov core. OV Core use count: 0
Created ov core.
Unexpected application crash with code: 11
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247057.xml_TargetDevice=TEMPLATE_Config=() returned/aborted with exit code -6 (8405 ms)
FAILED TESTS (1/1):
    8405 ms: /home/nsemaev/code/ConformanceTests/bin/conformanceTests_master_20

[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3297535.xml_TargetDevice=TEMPLATE_Config=() (15311 ms)
Note: Google Test filter = conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3297535.xml_TargetDevice=TEMPLATE_Config=()
[==========] Running 1 test from 1 test suite.
[----------] Global test environment set-up.
[----------] 1 test from conformance/ReadIRTest
[ RUN      ] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3297535.xml_TargetDevice=TEMPLATE_Config=()

MEM_USAGE=62488KB
Access PluginCache ov core. OV Core use count: 0
Created ov core.
Unexpected application crash with code: 11
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3297535.xml_TargetDevice=TEMPLATE_Config=() returned/aborted with exit code -6 (15311 ms)
FAILED TESTS (1/1):
   15311 ms: /home/nsemaev/code/ConformanceTests/bin/conformanceTests_master_2022_02_15 conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_3

[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247015.xml_TargetDevice=TEMPLATE_Config=() (26693 ms)
Note: Google Test filter = conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247015.xml_TargetDevice=TEMPLATE_Config=()
[==========] Running 1 test from 1 test suite.
[----------] Global test environment set-up.
[----------] 1 test from conformance/ReadIRTest
[ RUN      ] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247015.xml_TargetDevice=TEMPLATE_Config=()

MEM_USAGE=62488KB
Access PluginCache ov core. OV Core use count: 0
Created ov core.
Unexpected application crash with code: 11
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247015.xml_TargetDevice=TEMPLATE_Config=() returned/aborted with exit code -6 (26693 ms)
FAILED TESTS (1/1):
   26693 ms: /home/nsemaev/code/ConformanceTests/bin/conformanceTests_master_2022_02_15 conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2

[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247063.xml_TargetDevice=TEMPLATE_Config=() (31219 ms)
Note: Google Test filter = conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247063.xml_TargetDevice=TEMPLATE_Config=()
[==========] Running 1 test from 1 test suite.
[----------] Global test environment set-up.
[----------] 1 test from conformance/ReadIRTest
[ RUN      ] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247063.xml_TargetDevice=TEMPLATE_Config=()

MEM_USAGE=62488KB
Access PluginCache ov core. OV Core use count: 0
Created ov core.
Unexpected application crash with code: 11
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2247063.xml_TargetDevice=TEMPLATE_Config=() returned/aborted with exit code -6 (31219 ms)
FAILED TESTS (1/1):
   31219 ms: /home/nsemaev/code/ConformanceTests/bin/conformanceTests_master_2022_02_15 conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=BinaryConvolution_2

...

[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_2353441.xml_TargetDevice=TEMPLATE_Config=() (27400 ms)
2022_02_16 01:13:00 Reshape (86.02053165435791 of 240)
1689 of 1837
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_3373353.xml_TargetDevice=TEMPLATE_Config=() (39557 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_206524.xml_TargetDevice=TEMPLATE_Config=() (64229 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_2159272.xml_TargetDevice=TEMPLATE_Config=() (40266 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_3283854.xml_TargetDevice=TEMPLATE_Config=() (55205 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_789117.xml_TargetDevice=TEMPLATE_Config=() (47547 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_1534313.xml_TargetDevice=TEMPL

[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_789127.xml_TargetDevice=TEMPLATE_Config=() (85819 ms)
2022_02_16 01:13:20 Reshape (106.04223251342773 of 240)
1713 of 1837
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_1060951.xml_TargetDevice=TEMPLATE_Config=() (88486 ms)
Note: Google Test filter = conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_1060951.xml_TargetDevice=TEMPLATE_Config=()
[==========] Running 1 test from 1 test suite.
[----------] Global test environment set-up.
[----------] 1 test from conformance/ReadIRTest
[ RUN      ] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_1060951.xml_TargetDevice=TEMPLATE_Config=()

MEM_USAGE=62488KB
Access PluginCache ov core. OV Core use count: 0
Created ov core.
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_1060951.xml_TargetDevice=TEMPLATE_Config=() returned/aborted with exit code -9 (88486 ms)
FAILED TESTS (1/1):
   88486 ms: /home/nsemaev/cod

[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_206379.xml_TargetDevice=TEMPLATE_Config=() (98401 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_2667432.xml_TargetDevice=TEMPLATE_Config=() (124746 ms)
2022_02_16 01:13:40 Reshape (126.20019555091858 of 240)
1736 of 1837
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_2134871.xml_TargetDevice=TEMPLATE_Config=() (123944 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_3374037.xml_TargetDevice=TEMPLATE_Config=() (124988 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_402653.xml_TargetDevice=TEMPLATE_Config=() (94708 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_3284054.xml_TargetDevice=TEMPLATE_Config=() (76592 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_92350.xml_TargetDevice=TEM

[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_402678.xml_TargetDevice=TEMPLATE_Config=() (100327 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_3479754.xml_TargetDevice=TEMPLATE_Config=() (142207 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_80718.xml_TargetDevice=TEMPLATE_Config=() (146615 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_3466645.xml_TargetDevice=TEMPLATE_Config=() (131811 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_2667424.xml_TargetDevice=TEMPLATE_Config=() (127272 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_1302188.xml_TargetDevice=TEMPLATE_Config=() (106084 ms)
[0/1] Running tests...
[1/1] conformance/ReadIRTest.ReadIR/PRC=ir_IR_name=Reshape_2134919.xml_TargetDevice=TEMPLATE_Config=() (115039 ms)
[0/1] Running tests...
[1/1] conforma

End
DeformableConvolution_opset8
Interpolate_opset1
MVN_opset2
MaxPool_opset8


In [187]:
for op in all_ops:
    run_folders = [folder for folder in os.listdir(results_path) 
                   if re.findall(fr"{op}_.+_completed", folder)]
    if run_folders:
        run_folder = sorted(run_folders)[-1]
        good_tests, all_tests = re.findall(fr"{op}_.+_(\d+)_of_(\d+)_\d+s_completed", run_folder)[0]
        if good_tests != all_tests:
            print("TIME OUT: ", run_folder)
    
    

In [122]:
print(get_precision_irs_dict('Add'))

{'f16': [592109, 206486, 223903, 1345, 50056, 1269, 1395938, 1395935], 'f32': [15647, 69713, 42407, 712028, 579, 175242, 712025, 655], 'i64': [687724, 2240, 67575, 67590], 'i32': [42965, 1426552, 1767779, 1060907]}


In [7]:
os.system("$(pgrep -f conformanceTests)")

sh: 1: 69938: not found


32512

In [34]:
number = 4
data = {}
with open(report_file, 'r') as file:
    soup = BeautifulSoup(str(file.read()), 'html.parser')
tbody = soup.findAll('tbody')[1]
for op_tr in tbody.findAll('tr'):
    op = op_tr.findAll('th')[0].text.split('-')[0]
    template_td = op_tr.findAll('td')[number]
    print(op)
    data[op] = {result: 'untested' for result in ci_results}
    for result in data[op].keys():
        print(result)
        if len(template_td.find_all('span')) == 4:
            data[op][result] = int(re.findall(fr"{result[0].upper()}:(\d+)", template_td.text)[0])
print(data)
# return data

Abs
passed
failed
skipped
crashed
Acos
passed
failed
skipped
crashed
Acosh
passed
failed
skipped
crashed
AdaptiveAvgPool
passed
failed
skipped
crashed
AdaptiveMaxPool
passed
failed
skipped
crashed
Add
passed
failed
skipped
crashed
Asin
passed
failed
skipped
crashed
Asinh
passed
failed
skipped
crashed
Assign
passed
failed
skipped
crashed
Assign
passed
failed
skipped
crashed
Atan
passed
failed
skipped
crashed
Atanh
passed
failed
skipped
crashed
AvgPool
passed
failed
skipped
crashed
BatchNormInference
passed
failed
skipped
crashed
BatchNormInference
passed
failed
skipped
crashed
BatchToSpace
passed
failed
skipped
crashed
BinaryConvolution
passed
failed
skipped
crashed
Broadcast
passed
failed
skipped
crashed
Broadcast
passed
failed
skipped
crashed
Bucketize
passed
failed
skipped
crashed
CTCGreedyDecoder
passed
failed
skipped
crashed
CTCGreedyDecoderSeqLen
passed
failed
skipped
crashed
CTCLoss
passed
failed
skipped
crashed
Ceiling
passed
failed
skipped
crashed
Clamp
passed
failed
skipped
cr